# Pairwise correlation for robust feature selection

* Feature selection is important to remove redundant features, reduce model complexity, and avoid overfitting
* Some techniques focus on removal of features based on individual properties
* Pairwise correlation examines the connection between features and decides if they are worth keeping


### Pearson's correlation coefficient

* Quantifies the linear relationship between two distributions (or features) in a single metric
* Symbol is `r` with range [-1, 1]
  * -1 is a perfect negative correlation
  * +1 is a perfect positive correlation
  * strong correlation: 0.7 <= |r| < 1.0
  * moderate correlation: 0.3 <= |r| < 0.7
  * weak correlation: 0.0 <= |r| < 0.3


### How Pearson correlation is used in feature selection

* Let's say that the Pearson correlation between two feaures is 0.85
* This means that 85% of the time, you can predict feature 2 by using the values of feature 1
* If you have feature 1 in your dataset, feature 2 is not adding much information
* You can remove feature 2 since it only adds more complexity to our trained model
* Pairwise correlation identifies groups of highly correlated features and only keeps one of them
* The model has high predictive power using fewer features


### How to identify highly correlated features

* Plot a correlation matrix